In [2]:
###Differential expression analysis of PFC
## Andi Liu
# 1/9/2024

## loading
suppressPackageStartupMessages({
library(Seurat)
library(Signac)
library(dplyr)
library(patchwork)
library(future)
library(stringr)
library(tidydr)
library(tidyverse)
library(viridis)
library(qs)
library(ggplot2)
library(readxl)

library(RColorBrewer)
library(ComplexHeatmap)
library(circlize)

library(GenomicRanges)
library(GenomicFeatures)
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)
})
setwd("/data2/aliu8/2023_AD_multiome/Analysis/")

In [ ]:
### loading object
clean <- readRDS("03.clean_object_rmOutlier.rds")
ls()
#clean
meta <- clean@meta.data
head(meta)

In [40]:
#######################################################
## region umap ########################################
#######################################################
col1=c('#F06719','#33A65C','#23767C','#E03426','#1BA3C6',"#A26DC2","#FCB905","#EB73B3")
names(col1)=c('Astrocyte','Excitatory','Inhibitory','Microglia','Oligodendrocyte',"OPC","Endothelial","VLMC/Per")
cols=c(unname(col1),'grey90')
names(cols)=c(names(col1),'other')

In [ ]:
### Figure 1 panel b ###
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))

pdf(file='./Figures/UMAP/Regions/BrainRegion_UMAP_modality.pdf',width = 5,height = 5)
print(DimPlot(clean,reduction = 'wnn.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("ALL"))
print(DimPlot(clean,reduction = 'harmony.rna.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("RNA"))
print(DimPlot(clean,reduction = 'harmony.peaks.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("ATAC"))
dev.off()

In [8]:
################################################################################
############################# clean the metadata ###############################
################################################################################
# pull out metadata
biospecimen_meta <- read.csv("./Analysis/EOAD_biospecimen_metadata.csv")
meta <- clean@meta.data

# add phenotype information
id <- match(clean$individual_ID,biospecimen_meta$Simple_ID)

meta$sex <- as.factor(biospecimen_meta[id,]$Sex)
meta$age <- biospecimen_meta[id,]$Age
meta$race <- as.factor(biospecimen_meta[id,]$Race)
meta$ethinicity <- as.factor(biospecimen_meta[id,]$Ethinicity)
meta$PMI <- biospecimen_meta[id,]$PMI.Hours.
meta$RIN <- biospecimen_meta[id,]$New_RIN
meta$batch <- as.factor(meta$batch)
#meta$batch <- biospecimen_meta[id,]$Batch

#meta

## assign back
clean@meta.data <- meta

In [ ]:
### Figure 1 panel c
######################################################################
###########-- check cell type marker genes on each cluster --#########
######################################################################
Idents(clean) <- "cluster_celltype"
Idents(clean) <- factor(clean@active.ident, sort(levels(clean@active.ident),decreasing = T))

DefaultAssay(clean) <- "PC"

p1 <-  DotPlot(clean,cols = viridis(3, option = "plasma"),
             features = c("AQP4","GFAP","GJA1", # astrocytes
                          "FLT1","CLDN5","ITM2A",# endotheial
                                 #"SYT1",#neuron in general
                          "SLC17A7","NRGN","CAMK2A",# excitatory neurons
                          "GAD2","GAD1", # inhibitory neurons
                          "CSF1R","CD74","C3",# microglia
                          "MOBP","MOG","PLP1",# oligodendrocyte
                          "PDGFRA","VCAN","OLIG2", # OPCs
                          "PDGFRB","PDE5A" # VLMC/Pericyte
                                )) + RotatedAxis() +
                                    theme(axis.title = element_blank(),legend.position = "top")
ggsave(filename = "./Figures/Fig1/CT_features_dotplot.pdf",
    patchwork::wrap_plots(p1, ncol = 1),
      scale = 1, width = 9, height = 4)

In [18]:
### Figure 1 panel d
## marker_coverage_plot
DefaultAssay(clean)<-"ATAC"
Annotation(clean) <- annot
a<-FindRegion(clean, "AQP4"  ,extend.downstream=800,extend.upstream=100)
b<-FindRegion(clean, "SLC17A7" ,extend.downstream=500,extend.upstream=100)
c<-FindRegion(clean, "GAD2"  ,extend.downstream=100,extend.upstream=1000)
d<-FindRegion(clean, "CSF1R",extend.downstream=400,extend.upstream=100)
e<-FindRegion(clean, "MOBP"  ,extend.downstream=100,extend.upstream=1000)
f<-FindRegion(clean, "PDGFRA"  ,extend.downstream=100,extend.upstream=1000) 

In [ ]:
DefaultAssay(clean)<-"ATAC"

Annotation(clean)<-annot[which(annot$gene_name=="AQP4"),]
p1<-CoveragePlot(clean,window=400,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =a)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          #axis.line.y = element_blank(),
          strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("AQP4")

anPlot<-AnnotationPlot(clean, region=a)+ scale_color_manual(values="darkgreen")+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(),axis.ticks.x = element_blank(), 
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p1<-CombineTracks(list(p1,anPlot),heights=c(7,1))

Annotation(clean)<-annot[which(annot$gene_name=="SLC17A7"),]
p2<-CoveragePlot(clean,window=400,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =b)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          axis.line.y = element_blank(),strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("SLC17A7")

anPlot<-AnnotationPlot(clean, region=b)+ scale_color_manual(values="darkgreen")+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(), axis.line.y = element_blank(),axis.ticks.x = element_blank(),
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p2<-CombineTracks(list(p2,anPlot),heights=c(7,1))

Annotation(clean)<-annot[which(annot$gene_name=="GAD2"),]
p3<-CoveragePlot(clean,window=500,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =c)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          axis.line.y = element_blank(),strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("GAD2")

anPlot<-AnnotationPlot(clean, region=c)+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(), axis.line.y = element_blank(),axis.ticks.x = element_blank(),
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p3<-CombineTracks(list(p3,anPlot),heights=c(7,1))

Annotation(clean)<-annot[which(annot$gene_name=="CSF1R"),]
p4<-CoveragePlot(clean,window=500,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =d)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          axis.line.y = element_blank(),strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("CSF1R")

anPlot<-AnnotationPlot(clean, region=d)+scale_color_manual(values="darkgreen")+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(), axis.line.y = element_blank(),axis.ticks.x = element_blank(),
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p4<-CombineTracks(list(p4,anPlot),heights=c(7,1))

Annotation(clean)<-annot[which(annot$gene_name=="MOBP"),]
p5<-CoveragePlot(clean,window=600,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =e)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          axis.line.y = element_blank(),strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("MOBP")

anPlot<-AnnotationPlot(clean, region=e)+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(), axis.line.y = element_blank(),axis.ticks.x = element_blank(),
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p5<-CombineTracks(list(p5,anPlot),heights=c(7,1))

Annotation(clean)<-annot[which(annot$gene_name=="PDGFRA"),]
p6<-CoveragePlot(clean,window=800,idents=c("Astrocyte","Excitatory","Inhibitory","Microglia","Oligodendrocyte","OPC"),
                 group.by="cluster_celltype",annotation=F,peaks=F, region =f)+
    scale_fill_manual(values=c("#F06719","#33A65C","#23767C","#E03426","#EB73B3","#A26DC2"))+ 
    theme(strip.text.y.left = element_blank(),axis.ticks.y = element_blank(),
          axis.line.y = element_blank(),strip.background = element_blank(),plot.title = element_text(hjust = 0.5), axis.title.y=element_blank())+ggtitle("PDGFRA")

anPlot<-AnnotationPlot(clean, region=f)+
        theme(strip.text.y.left = element_blank(),strip.background = element_blank(), axis.line.y = element_blank(),axis.ticks.x = element_blank(),
              axis.text.x=element_blank(),axis.title.y=element_blank(),plot.title = element_text(hjust = 0.5))

p6<-CombineTracks(list(p6,anPlot),heights=c(7,1))

In [ ]:
library(cowplot)
pdf("./Figures/marker_coverage_plot.pdf", width=12, height=7)
patchwork::wrap_plots(p1,p2,p3,p4,p5,p6,ncol = 6)
dev.off()

In [ ]:
### Figure 1 panel e
### proportional analysis 
library(speckle)
meta <- clean@meta.data

## PFC
p1 <- plotCellTypeProps(clusters = meta[meta$regions == "PFC",]$cluster_celltype,
                        sample = meta[meta$regions == "PFC",]$individual_ID,)+ 
  theme(axis.text.x = element_text(angle = 45))+ ggtitle("Cell type proportions by PFC samples by cluster") + 
  theme(plot.title = element_text(size = 18, hjust = 0))
p1 <- p1 + theme_bw() + theme(panel.grid.major = element_blank(),
                        panel.grid.minor = element_blank()) + theme(axis.text.x = element_text(angle = 90))

## EC
p2 <- plotCellTypeProps(clusters = meta[meta$regions == "EC",]$major.cell.type.prediction.mathy,
                        sample = meta[meta$regions == "EC",]$individual_ID,)+ 
  theme(axis.text.x = element_text(angle = 45))+ ggtitle("Cell type proportions by EC samples by cluster") + 
  theme(plot.title = element_text(size = 18, hjust = 0))
p2 <- p2 + theme_bw() + theme(panel.grid.major = element_blank(),
                        panel.grid.minor = element_blank()) + theme(axis.text.x = element_text(angle = 90))
## HIP
p3 <- plotCellTypeProps(clusters = meta[meta$regions == "HIP",]$major.cell.type.prediction.mathy,
                        sample = meta[meta$regions == "HIP",]$individual_ID,)+ 
  theme(axis.text.x = element_text(angle = 45))+ ggtitle("Cell type proportions by HIP samples by cluster") + 
  theme(plot.title = element_text(size = 18, hjust = 0))
p3 <- p3 + theme_bw() + theme(panel.grid.major = element_blank(),
                        panel.grid.minor = element_blank()) + theme(axis.text.x = element_text(angle = 90))


## saving cell type proportional figure
library(cowplot)
pdf("./Figures/Cell_type_proportion_by_individual_new.pdf", width=12, height=5)
plot_grid(p1,p2,p3, ncol=3)
dev.off()
